<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [3]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [4]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [12]:
%%sql

SELECT capacity - coalesce(count(ticket_id), 0) as remaining
FROM screenings
JOIN theaters USING (theater_id)
JOIN tickets USING (screening_id)

 * sqlite:///movies.sqlite
Done.


remaining
236


In [4]:
%%sql

SELECT title FROM movies

 * sqlite:///movies.sqlite
Done.


title
Mr. Nobody
Forrest Gump
The Shawshank Redemption


+ Show the performance dates for one of the movies.

In [23]:
%%sql
SELECT start_time, title
FROM screenings
JOIN movies USING (imdb)
LIMIT 1


 * sqlite:///movies.sqlite
Done.


start_time,title
20220208 06:00:00 PM,Mr. Nobody


+ Show all data concerning performances at a given theatere
  on a given date.

In [23]:
%%sql
SELECT capacity - coalesce(count(ticket_id), 0) AS remaining
FROM theaters
JOIN screenings USING (theater_id)
JOIN tickets USING (screening_id)

 * sqlite:///movies.sqlite
Done.


remaining
236


+ List all customers

In [41]:
%%sql
SELECT * FROM customers

 * sqlite:///movies.sqlite
Done.


customer_id,name,username,password
fc1abb852916198b8f218f0e413b5f46,Elham,Elly,password
c136fdb998f3aa1fe752bdb9f2d303a1,Erik,CoolGuy69,password
064fb89f3b438645145d421d7a4dea36,Lars,Lassemannen,password


+ List all tickets

In [42]:
%%sql
SELECT * FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,screening_id,customer_id


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [69]:
%%sql

INSERT INTO tickets (screening_id, customer_id)
VALUES ((SELECT screening_id FROM screenings LIMIT 1), (SELECT customer_id FROM customers LIMIT 1));

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [70]:
%%sql
SELECT ticket_id, title, username, start_time, theater_name FROM tickets
JOIN customers USING (customer_id)
JOIN screenings USING (screening_id)
JOIN theaters USING (theater_id)
JOIN movies USING (imdb)


 * sqlite:///movies.sqlite
Done.


ticket_id,title,username,start_time,theater_name
603b7ab6c60f5514a7ac9eb819530feb,The Shawshank Redemption,Lassemannen,20220209 05:00:00 PM,Royal


+ Try to insert two movie theaters with the same name (this
  should fail).

In [80]:
%%sql
INSERT INTO theaters (theater_name, capacity)
VALUES
  ("Kino", 30),
  ("Kino", 123);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name
[SQL: INSERT INTO theaters (theater_name, capacity)
VALUES
  ("Kino", 30),
  ("Kino", 123);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [84]:
%%sql
INSERT INTO screenings (start_time, imdb, theater_id)
VALUES
  ("20220210 08:00 PM", "tt0485947", "potato");

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO screenings (start_time, imdb, theater_id)
VALUES
  ("20220210 08:00 PM", "tt0485947", "potato");]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [86]:
%%sql
INSERT INTO tickets (screening_id, customer_id)
VALUES
  ((SELECT screening_id FROM screenings LIMIT 1), "potato");

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets (screening_id, customer_id)
VALUES
  ((SELECT screening_id FROM screenings LIMIT 1), "potato");]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [88]:
%%sql
INSERT INTO tickets (screening_id, customer_id)
VALUES
  ("potato", (SELECT customer_id FROM customers LIMIT 1));

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets (screening_id, customer_id)
VALUES
  ("potato", (SELECT customer_id FROM customers LIMIT 1));]
(Background on this error at: https://sqlalche.me/e/14/gkpj)